#Import Required Methods

In [73]:
import pandas as pd
import numpy as np
from numpy import mean

import datetime as dt

import matplotlib.pyplot as plt

#Sklearn
from sklearn.model_selection import cross_val_score, train_test_split, RepeatedStratifiedKFold, cross_validate, GridSearchCV,\
StratifiedKFold
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.utils import resample, class_weight
from sklearn.metrics import precision_recall_curve, auc, make_scorer, average_precision_score, accuracy_score,\
recall_score, precision_score, f1_score, roc_curve, balanced_accuracy_score, roc_auc_score, classification_report, confusion_matrix,\
brier_score_loss

from imblearn.pipeline import Pipeline
from imblearn.under_sampling import RandomUnderSampler


In [7]:
#Imports for Neural Network.
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, applications, optimizers, regularizers
from keras.models import Sequential,Model
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier

In [94]:
# Access files from Google Drive

# Global Parameters, Options, and Variables

In [83]:
#Global parameter for setting randon_state in various methods.
rs = 0

In [8]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 200

In [9]:
#Numerical features used in modelling.
numModCol = ['acc_open_past_24mths', 'annual_inc', 'avg_cur_bal', 'bc_open_to_buy', 'credit_hist_months',
            'dti', 'fico', 'inq_last_6mths', 'installment', 'mo_sin_rcnt_rev_tl_op',
            'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'num_actv_rev_tl', 'num_tl_op_past_12m', 
            'open_acc', 'percent_bc_gt_75', 'pub_rec', 'revol_bal', 'total_rev_hi_lim', 'int_rate']


In [10]:
#Categorical features used in modelling.
catModCol = ['term', 'purpose', 'emp_length', 'home_ownership', 'verification_status', 'grade']

# Access files from Google Drive

In [2]:
#Get access to Google Drive where files are stored.
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True) 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
#Switch directory in Google Drive.
%cd /content/gdrive/My Drive/MRP/

/content/gdrive/My Drive/MRP


In [11]:
#Load pickled dataframe.
dfFinal = pd.read_pickle('dfFinalPickleColab.pkl')

In [12]:
#Dataframe with just the model features.
dfModel = dfFinal[numModCol+catModCol]

# Missing Values

In [13]:
#Columns with missing values and percentage of instances containing them that need to be addressed.
dfMissing = pd.DataFrame((dfModel.isna().sum() / dfModel.shape[0] * 100)[dfModel.isna().sum() > 0])
dfMissing.index.name = 'Feature'
dfMissing.columns = ['Instance %']
dfMissing = dfMissing.round(3)
dfMissing = dfMissing.sort_values(by = ['Instance %'])
dfMissing

,Instance %
Feature,
dti,0.004
acc_open_past_24mths,4.250
mort_acc,4.250
mths_since_recent_bc,5.171
bc_open_to_buy,5.239
percent_bc_gt_75,5.275
mo_sin_rcnt_rev_tl_op,6.142
mo_sin_rcnt_tl,6.142
num_actv_rev_tl,6.142


## Drop unused categorical labels in categorical features.

In [14]:
for col in catModCol:
    dfFinal.loc[:, col] = dfFinal.loc[:,col].cat.remove_unused_categories()
    

## Establishing Training and Testing Instances

In [15]:
#Percentage of records by term and year.
yearTermQty = dfFinal.groupby(['issue_d_year', 'term'], observed=True).size()
termQty = dfFinal.groupby(['term'], observed=True).size()

In [16]:
yearTermQty.div(termQty, axis='index')

issue_d_year  term     
2007          36 months    0.000259
2008          36 months    0.001417
2009          36 months    0.004980
2010          36 months    0.008554
              60 months    0.019774
2011          36 months    0.014904
              60 months    0.055729
2012          36 months    0.046153
              60 months    0.077711
2013          36 months    0.106669
              60 months    0.271018
2014          36 months    0.172686
              60 months    0.575768
2015          36 months    0.300794
2016          36 months    0.343583
dtype: float64

In [17]:
#Checking the breakdown of 60 month loans by month in 2014 to establish cut-off.
group1 = dfFinal[(dfFinal.issue_d_year == '2014') & (dfFinal.term_numeric==60)].groupby(['issue_d_month']).size()
totalQty = dfFinal[(dfFinal.issue_d_year == '2014') & (dfFinal.term_numeric==60)].shape[0]
np.cumsum(group1.div(totalQty))

issue_d_month
1          0.056575
2          0.115791
3          0.182550
4          0.262025
5          0.344033
6          0.421661
7          0.549854
8          0.625565
9          0.669847
10         0.836970
11         0.953062
12         1.000000
Missing    1.000000
dtype: float64

In [18]:
#Locations of 36 months training and testing data.
train36 = ((dfFinal.term_numeric==36) & (dfFinal.issue_d <= dt.datetime(year=2015, month =12, day=31)))
test36 = ((dfFinal.term_numeric==36) & (dfFinal.issue_d > dt.datetime(year=2015, month =12, day=31)))

In [19]:
#Locations of 60 months training and testing data.
train60 = ((dfFinal.term_numeric==60) & (dfFinal.issue_d <= dt.datetime(year = 2014, month = 6, day = 30)))
test60 = ((dfFinal.term_numeric==60) & (dfFinal.issue_d > dt.datetime(year = 2014, month = 6, day = 30)))

In [20]:
#Count of each type of loan based on term.
count36 = (dfFinal.term_numeric==36).sum()
count60 = (dfFinal.term_numeric==60).sum()

In [21]:
trainAll = (train36 | train60)
testAll = (test36 | test60)

In [22]:
trainTest36 = pd.Series([train36.sum()/count36, test36.sum()/count36], name = '36 Months')
trainTest60 = pd.Series([train60.sum()/count60, test60.sum()/count60], name = '60 Months')
trainTestAll = pd.Series([trainAll.sum() / dfFinal.shape[0], testAll.sum()/dfFinal.shape[0]], name = 'All Data')
df = (pd.DataFrame([trainTest36, trainTest60, trainTestAll]).T*100).round(1)
df.index = ['Train %', 'Test %']
df.T

,Train %,Test %
36 Months,65.6,34.4
60 Months,66.7,33.3
All Data,65.8,34.2


In [23]:
trainLabels = pd.get_dummies(dfFinal.loan_status)['Charged Off'][trainAll]
testLabels = pd.get_dummies(dfFinal.loan_status)['Charged Off'][testAll]

# Create Scalers for Numeric Features

In [29]:
scaler = StandardScaler()

In [30]:
scaler = scaler.fit(dfFinal.loc[trainAll, numModCol])

In [31]:
scaledArray = scaler.transform(dfFinal[numModCol])

In [32]:
dfScaled = pd.DataFrame(scaledArray, columns=numModCol, index =dfFinal[numModCol].index )

# Create one-hot encoded indicator columns for all the categorical features.

In [33]:
oneEncoder = OneHotEncoder()

In [34]:
oneHotArray = oneEncoder.fit_transform(dfFinal[catModCol])

In [35]:
oneHotColumns = oneEncoder.get_feature_names(catModCol)

In [36]:
dfFinalOne = pd.concat([dfScaled, pd.DataFrame(oneHotArray.toarray(), columns=oneHotColumns, index = dfScaled.index)], axis = 1)

In [37]:
#Excludes original categorical columns.
dfFinalOneLimit = dfFinalOne[numModCol + oneHotColumns.tolist()]
dfFinalOneLimit.head()

,acc_open_past_24mths,annual_inc,avg_cur_bal,bc_open_to_buy,credit_hist_months,dti,fico,inq_last_6mths,installment,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,num_actv_rev_tl,num_tl_op_past_12m,open_acc,percent_bc_gt_75,pub_rec,revol_bal,total_rev_hi_lim,int_rate,term_36 months,term_60 months,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,emp_length_Missing,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G
0,1.580095,-0.047358,2.309900,-0.498368,0.119843,-1.079649,0.637486,0.296605,-0.371608,0.340303,-0.568557,1.586125,-0.185791,-0.851168,2.335077,-0.248784,-1.385753,-0.333064,-0.660921,-0.699985,-0.281675,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-1.133706,-0.734901,-0.665976,-0.295089,1.518996,1.884512,0.637486,0.296605,-0.343091,-0.628604,-0.568557,-0.814465,-0.705763,0.093699,-0.598523,-0.248784,-0.446996,1.367555,-0.386778,-0.281345,0.077076,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,-1.472931,-0.624495,-0.560560,-0.331473,-0.634388,0.199341,0.960813,-0.719712,0.567444,1.793664,2.237010,-0.814465,0.594166,-0.851168,-1.185243,-1.212674,0.494581,-0.333064,0.011673,-0.270879,0.424025,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.223194,0.102885,-0.649397,0.983254,1.431549,0.290006,1.769133,-0.719712,-0.719057,-0.507490,-0.352744,-0.814465,-0.640766,1.353522,0.574917,0.522328,-0.579493,-0.333064,-0.224812,0.320449,-1.733198,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.901645,-0.740144,-0.548986,NaN,-0.929522,1.078192,1.769133,-0.719712,-1.023848,0.037520,-0.352744,-0.814465,NaN,-1.166123,0.574917,-1.019896,NaN,-0.333064,-0.430548,-0.370306,-0.451609,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [38]:
dfFinalOneLimit.shape

(1068298, 65)

# Get Missing Values from Imputed File

In [39]:
imputedArray = pd.read_csv('missingImputs.csv')
missingColumns=imputedArray.columns

In [40]:
dfFinalOneLimit.loc[:,missingColumns] = imputedArray.values

In [41]:
dfPostImpute = dfFinalOneLimit

# Sample for performing cross validation grid search

In [42]:
dfTrain, dfTest, yTrain, yTest = train_test_split(dfPostImpute[trainAll], trainLabels, stratify = trainLabels, train_size =0.7, random_state=rs)

# Functions

In [43]:
def calcCostRev(y, y_pred, cost = (1,4), returnThreshold = False):
    #y_true must be a single column of binary labels for positive class.
    #y_prob must be single column of probabilities for positive class.
    
    thresholds = [0.5]
    result = y_pred.reshape(-1,1) >=  np.array(thresholds)
    
    #Convert boolean result array into binary array.
    resultBinary = result.astype('int')
    
    #Calculate loss from False Negatives
    lossFN = np.sum((y.values.reshape(-1,1) > resultBinary), axis=0)*cost[1]
    
    #Calculate loss from False Positive
    lossFP = np.sum((y.values.reshape(-1,1) < resultBinary), axis=0)*cost[0]
    
    totalLoss = lossFN + lossFP
    
    minInd = np.argmin(totalLoss)
    minThres = thresholds[minInd]
    minLoss = totalLoss[minInd]
    
    if returnThreshold:
        return minLoss, minThres
    else:
        return minLoss

In [44]:
def calcCost(y, y_pred, cost = (1,4), returnThreshold = False):
    #y_true must be a single column of binary labels for positive class.
    #y_prob must be single column of probabilities for positive class.
    
    thresholds = np.arange(0, 1, 0.001)
    result = y_pred.reshape(-1,1) >=  np.array(thresholds)
    
    #Convert boolean result array into binary array.
    resultBinary = result.astype('int')
    
    #Calculate loss from False Negatives
    lossFN = np.sum((y.values.reshape(-1,1) > resultBinary), axis=0)*cost[1]
    
    #Calculate loss from False Positive
    lossFP = np.sum((y.values.reshape(-1,1) < resultBinary), axis=0)*cost[0]
    
    totalLoss = lossFN + lossFP
    
    minInd = np.argmin(totalLoss)
    minThres = thresholds[minInd]
    minLoss = totalLoss[minInd]
    
    if returnThreshold:
        return minLoss, minThres
    else:
        return minLoss

In [45]:
def getLoss(y, y_hat, cost = (1,5)):
    #Calculate loss from False Negatives
    lossFN = np.sum(y > y_hat)*cost[1]
    
    #Calculate loss from False Positive
    lossFP = np.sum(y < y_hat)*cost[0]
    
    return lossFN+lossFP

In [46]:
predictLoss = make_scorer(score_func=calcCost, greater_is_better=False, needs_proba=True, cost = (1,4), returnThreshold = False)

In [47]:
def getFScore(y, y_prob):
    prec, recl, thres = precision_recall_curve(y, y_prob)
    fscore = (2 * prec * recl) / (prec + recl)
    maxInd = np.nanargmax(fscore)
    return fscore[maxInd]

In [48]:
highFScore = make_scorer(score_func=getFScore, greater_is_better=True, needs_proba=True)

In [49]:
def plot_roc(y_true, y_prob, plt, labelName):
    fpr, tpr, thresholds = roc_curve(y_true, y_prob)
    aucScore = np.round(roc_auc_score(y_true, y_prob),3)
    plt.plot(fpr, tpr, label = labelName + '- AUC: '+str(aucScore))
    plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5))
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

In [50]:
def plot_pre_recl(y_true, y_prob, plt, labelName):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    plt.plot(recall, precision, label = labelName)
    plt.legend(loc = 'center left', bbox_to_anchor=(1, 0.5))
    plt.xlabel('Recall')
    plt.ylabel('Precision')

In [51]:
def getScores(y_true, y_pred, y_prob):
    scoresDict ={}
    scoresDict['Accuracy'] = accuracy_score(y_true, y_pred)
    scoresDict['Precision(+)'] = precision_score(y_true, y_pred)
    scoresDict['Recall(+)'] = recall_score(y_true, y_pred)
    scoresDict['AUC'] = roc_auc_score(y_true, y_prob)
    scoresDict['Error Cost'] = getLoss(y_true, y_pred)
    return pd.DataFrame(scoresDict, index=['Result'])
    

In [52]:
def to_labels(origProb, threshold):
    result = origProb >= threshold
    return result.astype('int')

In [53]:
def fScoreThres(y, y_prob):
    prec, recl, thres = precision_recall_curve(y, y_prob)
    fscore = (2 * prec * recl) / (prec + recl)
    maxInd = np.argmax(fscore)
    return fscore[maxInd], thres[maxInd]

In [54]:
def aucPreRec(y, y_prob):
    prec, recl, thres = precision_recall_curve(y, y_prob)
    aucScore = auc(prec, recl)
    return aucScore

In [55]:
def plotGridScores(gridObj):
    allResults = {}
    for score in gridObj.scorer_.keys():
        allResults[score] = gridObj.cv_results_['mean_test_'+score].tolist()
    pd.DataFrame.from_dict(allResults).plot()

#Global Parameters for Grid Search

In [56]:
scoringDict2 = {'accuracy': make_scorer(accuracy_score), 
                'balanced_accuracy': make_scorer(balanced_accuracy_score),
               'f1': make_scorer(f1_score),
              'average_precision': make_scorer(average_precision_score, needs_proba=True), 
               'roc_auc': make_scorer(roc_auc_score, needs_proba=True), 
               'loan_loss': make_scorer(score_func=calcCostRev, greater_is_better=False, needs_proba=True, cost = (1,3.7), returnThreshold = False),
               'neg_brier_score': make_scorer(brier_score_loss, needs_proba=True)}

In [57]:
kFold = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

# Neural Network

In [84]:
#AUC metric for Keras neural net
auc = tf.keras.metrics.AUC(
    num_thresholds=200, curve='ROC', summation_method='interpolation', name=None,
    dtype=None, thresholds=None, multi_label=False, label_weights=None
)

In [118]:
def create_model1(actF = 'tanh', optim='rmsprop'):
  NN = Sequential()
  NN.add(layers.Dense(64, input_dim=dfPostImpute.shape[1], activation=actF))
  NN.add(Dropout(0.2))
  NN.add(layers.Dense(32, activation=actF))
  NN.add(Dropout(0.2))
  NN.add(layers.Dense(1, activation='sigmoid'))
  NN.compile(loss='binary_crossentropy', optimizer=optim, metrics=[auc, 'accuracy'])
  return NN

In [119]:
def create_model2(actF = 'tanh', optim='rmsprop'):
  NN = Sequential()
  NN.add(layers.Dense(32, input_dim=dfPostImpute.shape[1], activation=actF))
  NN.add(layers.Dense(16, activation=actF))
  NN.add(layers.Dense(1, activation='sigmoid'))
  NN.compile(loss='binary_crossentropy', optimizer=optim, metrics=[auc, 'accuracy'])
  return NN

In [120]:
def create_model3(actF = 'tanh', optim='rmsprop'):
  NN = Sequential()
  NN.add(layers.Dense(32, input_dim=dfPostImpute.shape[1], activation=actF))
  NN.add(layers.Dense(1, activation='sigmoid'))
  NN.compile(loss='binary_crossentropy', optimizer=optim, metrics=[auc, 'accuracy'])
  return NN

In [88]:
X_train = dfTrain
y_train = yTrain
X_test = dfTest
y_test = yTest

In [89]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(trainLabels), trainLabels )
d_class_weights = dict(enumerate(class_weights.tolist()))

# NN - Grid 1 - epochs, batch_size, and activation function

In [70]:
#Setup pipeline.
stepsNN = [('model', KerasClassifier(build_fn=create_model3, verbose=2))]
pipelineNN = Pipeline(steps=stepsNN)

In [71]:
#Parameters for grid search.
param_gridNN = [{'model__epochs': [50, 75, 100, 125],
              'model__batch_size': [1000, 5000, 10000, 15000, 20000],
              'model__actF': ['tanh', 'relu']}]

In [ ]:
#Grid search object
gridNN_1 = GridSearchCV(pipelineNN, param_grid=param_gridNN, scoring=scoringDict2, cv = kFold2, verbose=2,
                                        refit='roc_auc', n_jobs=-1)

In [ ]:
#Perform cross validation with grid search object.
gridNN_1.fit(X=dfPostImpute[trainAll], y=trainLabels)

Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 48.0min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 116.4min finished


Epoch 1/100
47/47 - 0s - loss: 0.6269 - auc: 0.5126 - accuracy: 0.6509
Epoch 2/100
47/47 - 0s - loss: 0.4249 - auc: 0.6169 - accuracy: 0.8463
Epoch 3/100
47/47 - 0s - loss: 0.4068 - auc: 0.6636 - accuracy: 0.8465
Epoch 4/100
47/47 - 0s - loss: 0.4031 - auc: 0.6759 - accuracy: 0.8464
Epoch 5/100
47/47 - 0s - loss: 0.4014 - auc: 0.6812 - accuracy: 0.8465
Epoch 6/100
47/47 - 0s - loss: 0.4003 - auc: 0.6845 - accuracy: 0.8466
Epoch 7/100
47/47 - 0s - loss: 0.3995 - auc: 0.6866 - accuracy: 0.8466
Epoch 8/100
47/47 - 0s - loss: 0.3989 - auc: 0.6883 - accuracy: 0.8466
Epoch 9/100
47/47 - 0s - loss: 0.3984 - auc: 0.6895 - accuracy: 0.8467
Epoch 10/100
47/47 - 0s - loss: 0.3980 - auc: 0.6906 - accuracy: 0.8467
Epoch 11/100
47/47 - 0s - loss: 0.3977 - auc: 0.6914 - accuracy: 0.8467
Epoch 12/100
47/47 - 0s - loss: 0.3974 - auc: 0.6922 - accuracy: 0.8467
Epoch 13/100
47/47 - 0s - loss: 0.3972 - auc: 0.6928 - accuracy: 0.8468
Epoch 14/100
47/47 - 0s - loss: 0.3969 - auc: 0.6935 - accuracy: 0.8468
E

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('model',
                                        <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f8f06878b38>)],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'model__actF': ['tanh', 'relu'],
                          'model__batch_size': [1000, 5000, 10000, 15000...
                      'average_precision': make_scorer(average_precision_score, needs_proba=True),
                      'balanced_accuracy': make_scorer(balanced_accuracy_score),
                      'f1': make_scorer(f1_score),
                      'loan_loss': make_scorer(calcCostRev, greater_is_better=False, needs_proba=True, cost=(1, 3.7), returnThreshold=False),
                      'neg_brier_score': make_scorer(brier_score_loss, 

In [ ]:
#Assign results of grid search.
results_Grid1_NN = gridNN_1.cv_results_

In [ ]:
#Best parameters based on AUC:
ind = np.argmin(results_Grid1_NN['rank_test_roc_auc'])
print('Parameters:', results_Grid1_NN['params'][ind])
print('ROC-AUC:', results_Grid1_NN['mean_test_roc_auc'][ind])
print('F1:', results_Grid1_NN['mean_test_f1'][ind])
print('Accuracy:', results_Grid1_NN['mean_test_accuracy'][ind])
print('Balanced Accuracy:', results_Grid1_NN['mean_test_balanced_accuracy'][ind])
print('Loan Loss:', results_Grid1_NN['mean_test_loan_loss'][ind])

Parameters: {'model__actF': 'relu', 'model__batch_size': 15000, 'model__epochs': 100}
ROC-AUC: 0.6989683507867918
F1: 0.01931538273595983
Accuracy: 0.8468088922576699
Balanced Accuracy: 0.5040215219973407
Loan Loss: -131777.4


In [ ]:
#Best parameters based on loan loss:
ind = np.argmin(results_Grid1_NN['rank_test_loan_loss'])
print('Parameters:', results_Grid1_NN['params'][ind])
print('ROC-AUC:', results_Grid1_NN['mean_test_roc_auc'][ind])
print('F1:', results_Grid1_NN['mean_test_f1'][ind])
print('Accuracy:', results_Grid1_NN['mean_test_accuracy'][ind])
print('Balanced Accuracy:', results_Grid1_NN['mean_test_balanced_accuracy'][ind])
print('Loan Loss:', results_Grid1_NN['mean_test_loan_loss'][ind])

Parameters: {'model__actF': 'tanh', 'model__batch_size': 1000, 'model__epochs': 75}
ROC-AUC: 0.6977938893079347
F1: 0.02852222383393682
Accuracy: 0.8465612380342035
Balanced Accuracy: 0.5058733318779411
Loan Loss: -131362.9



# NN - Grid 2 - Undersampling and optimizer.

In [ ]:
#Setup pipeline.
stepsNN = [('under', RandomUnderSampler()), ('model', KerasClassifier(build_fn=create_model3, verbose=2))]
pipelineNN = Pipeline(steps=stepsNN)

In [72]:
#Parameters for gridsearch.
param_gridNN = [{'model__epochs': [75],
              'model__batch_size': [1000],
              'under': [None,
                        RandomUnderSampler(sampling_strategy=0.2, random_state=rs),
                        RandomUnderSampler(sampling_strategy=0.6, random_state=rs),
                        RandomUnderSampler(sampling_strategy=1.0, random_state=rs)],
                 'model__optim': ['adam', 'adadelta', 'adagrad', 'rmsprop']}]

In [ ]:
#Grid search object.
gridNN = GridSearchCV(pipelineNN, param_grid=param_gridNN, scoring=scoringDict2, cv = kFold2, verbose=2,
                                        refit='roc_auc', n_jobs=-1, pre_dispatch=1)

In [ ]:
#Perform cross validation using grid search.
gridNN.fit(X=dfPostImpute[trainAll], y=trainLabels)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed: 50.3min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed: 58.4min finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/75
646/646 - 2s - loss: 0.4247 - auc: 0.6771 - accuracy: 0.8306
Epoch 2/75
646/646 - 2s - loss: 0.4176 - auc: 0.6923 - accuracy: 0.8334
Epoch 3/75
646/646 - 2s - loss: 0.4171 - auc: 0.6938 - accuracy: 0.8334
Epoch 4/75
646/646 - 2s - loss: 0.4165 - auc: 0.6951 - accuracy: 0.8335
Epoch 5/75
646/646 - 2s - loss: 0.4161 - auc: 0.6963 - accuracy: 0.8334
Epoch 6/75
646/646 - 2s - loss: 0.4158 - auc: 0.6969 - accuracy: 0.8335
Epoch 7/75
646/646 - 2s - loss: 0.4155 - auc: 0.6978 - accuracy: 0.8335
Epoch 8/75
646/646 - 2s - loss: 0.4152 - auc: 0.6984 - accuracy: 0.8335
Epoch 9/75
646/646 - 2s - loss: 0.4149 - auc: 0.6990 - accuracy: 0.8335
Epoch 10/75
646/646 - 2s - loss: 0.4147 - auc: 0.6997 - accuracy: 0.8335
Epoch 11/75
646/646 - 2s - loss: 0.4145 - auc: 0.7002 - accuracy: 0.8336
Epoch 12/75
646/646 - 2s - loss: 0.4144 - auc: 0.7005 - accuracy: 0.8335
Epoch 13/75
646/646 - 2s - loss: 0.4143 - auc: 0.7007 - accuracy: 0.8335
Epoch 14/75
646/646 - 2s - loss: 0.4141 - auc: 0.7012 - accu

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('under',
                                        RandomUnderSampler(random_state=None,
                                                           ratio=None,
                                                           replacement=False,
                                                           return_indices=False,
                                                           sampling_strategy='auto')),
                                       ('model',
                                        <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd4d3836668>)],
                                verb...
                      'average_precision': make_scorer(average_precision_score, needs_proba=True),
                      'balanced_accuracy': make_scorer(balanced_accuracy_score),
     

In [ ]:
#Assign results of gridsearch
results_Grid2_NN_2 = gridNN.cv_results_

In [ ]:
#Best parameters based on AUC:
ind = np.argmin(results_Grid2_NN_2['rank_test_roc_auc'])
print('Parameters:', results_Grid2_NN_2['params'][ind])
print('ROC-AUC:', results_Grid2_NN_2['mean_test_roc_auc'][ind])
print('F1:', results_Grid2_NN_2['mean_test_f1'][ind])
print('Accuracy:', results_Grid2_NN_2['mean_test_accuracy'][ind])
print('Balanced Accuracy:', results_Grid2_NN_2['mean_test_balanced_accuracy'][ind])
print('Loan Loss:', results_Grid2_NN_2['mean_test_loan_loss'][ind])

Parameters: {'model__batch_size': 1000, 'model__epochs': 75, 'model__optim': 'adam', 'under': RandomUnderSampler(random_state=0, ratio=None, replacement=False,
                   return_indices=False, sampling_strategy=0.2)}
ROC-AUC: 0.6977989876477055
F1: 0.04392049856764455
Accuracy: 0.8463135839748256
Balanced Accuracy: 0.5091485043524085
Loan Loss: -130611.8


In [ ]:
#Best parameters based on Loan Loss:
ind = np.argmin(results_Grid2_NN_2['rank_test_loan_loss'])
print('Parameters:', results_Grid2_NN_2['params'][ind])
print('ROC-AUC:', results_Grid2_NN_2['mean_test_roc_auc'][ind])
print('F1:', results_Grid2_NN_2['mean_test_f1'][ind])
print('Accuracy:', results_Grid2_NN_2['mean_test_accuracy'][ind])
print('Balanced Accuracy:', results_Grid2_NN_2['mean_test_balanced_accuracy'][ind])
print('Loan Loss:', results_Grid2_NN_2['mean_test_loan_loss'][ind])

Parameters: {'model__batch_size': 1000, 'model__epochs': 75, 'model__optim': 'rmsprop', 'under': RandomUnderSampler(random_state=0, ratio=None, replacement=False,
                   return_indices=False, sampling_strategy=0.6)}
ROC-AUC: 0.6966796492083605
F1: 0.3378787973025011
Accuracy: 0.7598267780900327
Balanced Accuracy: 0.6130396371842498
Loan Loss: -114222.40000000001


# NN - Grid 3 - Fine tuning Undersampler.
---



In [ ]:
#Setup pipeline.
stepsNN = [('under', RandomUnderSampler(random_state=rs, sampling_strategy=0.6)), ('model', KerasClassifier(build_fn=create_model3, verbose=0))]
pipelineNN = Pipeline(steps=stepsNN)

In [ ]:
#Parameters for gridsearch.
param_gridNN = [{'model__epochs': [75],
              'model__batch_size': [1000],
              'under': [RandomUnderSampler(sampling_strategy=0.3, random_state=rs),
                        RandomUnderSampler(sampling_strategy=0.4, random_state=rs),
                        RandomUnderSampler(sampling_strategy=0.5, random_state=rs),
                        RandomUnderSampler(sampling_strategy=0.6, random_state=rs),
                        RandomUnderSampler(sampling_strategy=0.7, random_state=rs),
                        RandomUnderSampler(sampling_strategy=0.8, random_state=rs),
                        RandomUnderSampler(sampling_strategy=0.9, random_state=rs)],
                 'model__optim': ['rmsprop']}]

In [ ]:
#Grid search object.
gridNN = GridSearchCV(pipelineNN, param_grid=param_gridNN, scoring=scoringDict2, cv = kFold2, verbose=0,
                                        refit='roc_auc', n_jobs=1)

In [ ]:
#Cross validation using grid search.
gridNN.fit(X=dfPostImpute[trainAll], y=trainLabels)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Instructions for updating:
Please use `model.predict()` instead.


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('under',
                                        RandomUnderSampler(random_state=0,
                                                           ratio=None,
                                                           replacement=False,
                                                           return_indices=False,
                                                           sampling_strategy=0.6)),
                                       ('model',
                                        <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fd4d3835e80>)],
                                verbose=Fa...
                      'average_precision': make_scorer(average_precision_score, needs_proba=True),
                      'balanced_accuracy': make_scorer(balanced_accuracy_score),
     

In [ ]:
#Assign results of grid search.
results_Grid3_NN = gridNN.cv_results_

In [ ]:
#Best parameters based on AUC:
ind = np.argmin(results_Grid3_NN['rank_test_roc_auc'])
print('Parameters:', results_Grid3_NN['params'][ind])
print('ROC-AUC:', results_Grid3_NN['mean_test_roc_auc'][ind])
print('F1:', results_Grid3_NN['mean_test_f1'][ind])
print('Accuracy:', results_Grid3_NN['mean_test_accuracy'][ind])
print('Balanced Accuracy:', results_Grid3_NN['mean_test_balanced_accuracy'][ind])
print('Loan Loss:', results_Grid3_NN['mean_test_loan_loss'][ind])

Parameters: {'model__batch_size': 1000, 'model__epochs': 75, 'model__optim': 'rmsprop', 'under': RandomUnderSampler(random_state=0, ratio=None, replacement=False,
                   return_indices=False, sampling_strategy=0.3)}
ROC-AUC: 0.6973125094948979
F1: 0.13513223141187106
Accuracy: 0.8408566548008514
Balanced Accuracy: 0.5297507172100174
Loan Loss: -126255.8


In [ ]:
#Best parameters based on loan loss:
ind = np.argmin(results_Grid3_NN['rank_test_loan_loss'])
print('Parameters:', results_Grid3_NN['params'][ind])
print('ROC-AUC:', results_Grid3_NN['mean_test_roc_auc'][ind])
print('F1:', results_Grid3_NN['mean_test_f1'][ind])
print('Accuracy:', results_Grid3_NN['mean_test_accuracy'][ind])
print('Balanced Accuracy:', results_Grid3_NN['mean_test_balanced_accuracy'][ind])
print('Loan Loss:', results_Grid3_NN['mean_test_loan_loss'][ind])

Parameters: {'model__batch_size': 1000, 'model__epochs': 75, 'model__optim': 'rmsprop', 'under': RandomUnderSampler(random_state=0, ratio=None, replacement=False,
                   return_indices=False, sampling_strategy=0.7)}
ROC-AUC: 0.6964972230277678
F1: 0.35104766148273847
Accuracy: 0.727946329191048
Balanced Accuracy: 0.6265844125943969
Loan Loss: -114034.23333333334


# NN: Differemt Architectures
---



In [ ]:
#First architecture
cvModel1 = cross_validate(estimator=KerasClassifier(build_fn=create_model1, verbose=2, optim = 'rmsprop'), X=dfPostImpute[trainAll], y=trainLabels, scoring=scoringDict2, cv = kFold,
                          fit_params = dict(batch_size=1000, epochs=75))

Epoch 1/75
469/469 - 2s - loss: 0.4120 - auc: 0.6920 - accuracy: 0.8421
Epoch 2/75
469/469 - 2s - loss: 0.4016 - auc: 0.6806 - accuracy: 0.8465
Epoch 3/75
469/469 - 2s - loss: 0.4003 - auc: 0.6842 - accuracy: 0.8465
Epoch 4/75
469/469 - 2s - loss: 0.3995 - auc: 0.6867 - accuracy: 0.8466
Epoch 5/75
469/469 - 2s - loss: 0.3993 - auc: 0.6876 - accuracy: 0.8467
Epoch 6/75
469/469 - 2s - loss: 0.3990 - auc: 0.6884 - accuracy: 0.8468
Epoch 7/75
469/469 - 2s - loss: 0.3985 - auc: 0.6899 - accuracy: 0.8467
Epoch 8/75
469/469 - 2s - loss: 0.3985 - auc: 0.6900 - accuracy: 0.8467
Epoch 9/75
469/469 - 2s - loss: 0.3983 - auc: 0.6906 - accuracy: 0.8467
Epoch 10/75
469/469 - 2s - loss: 0.3980 - auc: 0.6913 - accuracy: 0.8468
Epoch 11/75
469/469 - 2s - loss: 0.3978 - auc: 0.6916 - accuracy: 0.8468
Epoch 12/75
469/469 - 2s - loss: 0.3978 - auc: 0.6919 - accuracy: 0.8467
Epoch 13/75
469/469 - 2s - loss: 0.3980 - auc: 0.6912 - accuracy: 0.8469
Epoch 14/75
469/469 - 2s - loss: 0.3976 - auc: 0.6923 - accu

In [ ]:
#Second architecture
cvModel2 = cross_validate(estimator=KerasClassifier(build_fn=create_model2, verbose=2, optim = 'rmsprop'), X=dfPostImpute[trainAll], y=trainLabels, scoring=scoringDict2, cv = kFold,
                fit_params = dict(batch_size=1000,
                                  epochs=75))

Epoch 1/75
469/469 - 2s - loss: 0.4078 - auc: 0.6857 - accuracy: 0.8426
Epoch 2/75
469/469 - 2s - loss: 0.3976 - auc: 0.6923 - accuracy: 0.8467
Epoch 3/75
469/469 - 2s - loss: 0.3968 - auc: 0.6942 - accuracy: 0.8469
Epoch 4/75
469/469 - 2s - loss: 0.3962 - auc: 0.6958 - accuracy: 0.8469
Epoch 5/75
469/469 - 2s - loss: 0.3958 - auc: 0.6968 - accuracy: 0.8469
Epoch 6/75
469/469 - 2s - loss: 0.3954 - auc: 0.6979 - accuracy: 0.8470
Epoch 7/75
469/469 - 2s - loss: 0.3951 - auc: 0.6985 - accuracy: 0.8470
Epoch 8/75
469/469 - 2s - loss: 0.3949 - auc: 0.6993 - accuracy: 0.8470
Epoch 9/75
469/469 - 2s - loss: 0.3947 - auc: 0.6997 - accuracy: 0.8470
Epoch 10/75
469/469 - 2s - loss: 0.3945 - auc: 0.7003 - accuracy: 0.8469
Epoch 11/75
469/469 - 2s - loss: 0.3943 - auc: 0.7006 - accuracy: 0.8470
Epoch 12/75
469/469 - 2s - loss: 0.3942 - auc: 0.7010 - accuracy: 0.8470
Epoch 13/75
469/469 - 2s - loss: 0.3941 - auc: 0.7014 - accuracy: 0.8470
Epoch 14/75
469/469 - 2s - loss: 0.3939 - auc: 0.7016 - accu

In [ ]:
#Third architecture
cvModel3 = cross_validate(estimator=KerasClassifier(build_fn=create_model3, verbose=2, optim = 'rmsprop'), X=dfPostImpute[trainAll], y=trainLabels, scoring=scoringDict2, cv = kFold,
                fit_params = dict(batch_size=1000,
                                  epochs=75))

Epoch 1/75
469/469 - 2s - loss: 0.4078 - auc: 0.6925 - accuracy: 0.8425
Epoch 2/75
469/469 - 2s - loss: 0.3978 - auc: 0.6911 - accuracy: 0.8467
Epoch 3/75
469/469 - 2s - loss: 0.3974 - auc: 0.6924 - accuracy: 0.8468
Epoch 4/75
469/469 - 2s - loss: 0.3969 - auc: 0.6936 - accuracy: 0.8468
Epoch 5/75
469/469 - 2s - loss: 0.3966 - auc: 0.6945 - accuracy: 0.8469
Epoch 6/75
469/469 - 2s - loss: 0.3963 - auc: 0.6954 - accuracy: 0.8468
Epoch 7/75
469/469 - 2s - loss: 0.3961 - auc: 0.6959 - accuracy: 0.8468
Epoch 8/75
469/469 - 2s - loss: 0.3959 - auc: 0.6965 - accuracy: 0.8469
Epoch 9/75
469/469 - 2s - loss: 0.3956 - auc: 0.6971 - accuracy: 0.8469
Epoch 10/75
469/469 - 2s - loss: 0.3954 - auc: 0.6977 - accuracy: 0.8469
Epoch 11/75
469/469 - 2s - loss: 0.3953 - auc: 0.6980 - accuracy: 0.8470
Epoch 12/75
469/469 - 2s - loss: 0.3951 - auc: 0.6985 - accuracy: 0.8469
Epoch 13/75
469/469 - 2s - loss: 0.3949 - auc: 0.6989 - accuracy: 0.8470
Epoch 14/75
469/469 - 2s - loss: 0.3948 - auc: 0.6995 - accu

In [ ]:
cvModel1['test_roc_auc'].mean(), cvModel1['test_loan_loss'].mean()

(0.6990771527933791, -132348.06666666668)

In [ ]:
cvModel2['test_roc_auc'].mean(), cvModel2['test_loan_loss'].mean()

(0.695923159097911, -131294.23333333337)

In [ ]:
cvModel3['test_roc_auc'].mean(), cvModel3['test_loan_loss'].mean()

(0.6973859938966679, -131773.0)

# NN: under sampling strategy with final architecture

---



In [90]:
#Setup pipeline.
stepsNN = [('under', RandomUnderSampler()), ('model', KerasClassifier(build_fn=create_model2, verbose=2))]
pipelineNN = Pipeline(steps=stepsNN)

In [91]:
#Parameters for grid search.
param_gridNN = [{'model__epochs': [75],
              'model__batch_size': [1000],
              'under': [None,
                        RandomUnderSampler(random_state=rs, sampling_strategy=0.4),
                        RandomUnderSampler(random_state=rs, sampling_strategy=0.5),
                        RandomUnderSampler(random_state=rs, sampling_strategy=0.6),
                        RandomUnderSampler(random_state=rs, sampling_strategy=0.7),
                        RandomUnderSampler(random_state=rs, sampling_strategy=0.8),
                        RandomUnderSampler(random_state=rs, sampling_strategy=0.9),
                        RandomUnderSampler(random_state=rs, sampling_strategy=1)]}]


In [92]:
#Cross validation object.
gridNNFinal = GridSearchCV(pipelineNN, param_grid=param_gridNN, scoring=scoringDict2, cv = kFold2, verbose=1,
                                        refit='roc_auc', n_jobs=1)

In [93]:
#Fit cross validation object.
gridNNFinal.fit(X=dfPostImpute[trainAll], y=trainLabels)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/75
469/469 - 2s - loss: 0.4057 - auc_2: 0.6746 - accuracy: 0.8434
Epoch 2/75
469/469 - 2s - loss: 0.3976 - auc_2: 0.6919 - accuracy: 0.8467
Epoch 3/75
469/469 - 2s - loss: 0.3967 - auc_2: 0.6943 - accuracy: 0.8468
Epoch 4/75
469/469 - 2s - loss: 0.3962 - auc_2: 0.6957 - accuracy: 0.8469
Epoch 5/75
469/469 - 2s - loss: 0.3958 - auc_2: 0.6967 - accuracy: 0.8469
Epoch 6/75
469/469 - 2s - loss: 0.3955 - auc_2: 0.6976 - accuracy: 0.8469
Epoch 7/75
469/469 - 2s - loss: 0.3953 - auc_2: 0.6981 - accuracy: 0.8469
Epoch 8/75
469/469 - 2s - loss: 0.3950 - auc_2: 0.6989 - accuracy: 0.8469
Epoch 9/75
469/469 - 2s - loss: 0.3948 - auc_2: 0.6994 - accuracy: 0.8468
Epoch 10/75
469/469 - 2s - loss: 0.3947 - auc_2: 0.6998 - accuracy: 0.8470
Epoch 11/75
469/469 - 2s - loss: 0.3944 - auc_2: 0.7005 - accuracy: 0.8469
Epoch 12/75
469/469 - 2s - loss: 0.3943 - auc_2: 0.7008 - accuracy: 0.8470
Epoch 13/75
469/469 - 2s - loss: 0.3942 - auc_2: 0.7012 - accuracy: 0.8470
Epoch 14/75
469/469 - 2s - loss: 0

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/75
252/252 - 1s - loss: 0.5572 - auc_2: 0.7217 - accuracy: 0.7178
Epoch 2/75
252/252 - 1s - loss: 0.5512 - auc_2: 0.6901 - accuracy: 0.7222
Epoch 3/75
252/252 - 1s - loss: 0.5502 - auc_2: 0.6919 - accuracy: 0.7226
Epoch 4/75
252/252 - 1s - loss: 0.5495 - auc_2: 0.6931 - accuracy: 0.7234
Epoch 5/75
252/252 - 1s - loss: 0.5489 - auc_2: 0.6942 - accuracy: 0.7237
Epoch 6/75
252/252 - 1s - loss: 0.5484 - auc_2: 0.6950 - accuracy: 0.7240
Epoch 7/75
252/252 - 1s - loss: 0.5480 - auc_2: 0.6957 - accuracy: 0.7240
Epoch 8/75
252/252 - 1s - loss: 0.5476 - auc_2: 0.6964 - accuracy: 0.7242
Epoch 9/75
252/252 - 1s - loss: 0.5472 - auc_2: 0.6971 - accuracy: 0.7249
Epoch 10/75
252/252 - 1s - loss: 0.5468 - auc_2: 0.6977 - accuracy: 0.7245
Epoch 11/75
252/252 - 1s - loss: 0.5465 - auc_2: 0.6983 - accuracy: 0.7248
Epoch 12/75
252/252 - 1s - loss: 0.5463 - auc_2: 0.6987 - accuracy: 0.7248
Epoch 13/75
252/252 - 1s - loss: 0.5460 - auc_2: 0.6993 - accuracy: 0.7249
Epoch 14/75
252/252 - 1s - loss: 0

...

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/75
144/144 - 1s - loss: 0.6423 - auc_2: 0.6956 - accuracy: 0.6284
Epoch 2/75
144/144 - 0s - loss: 0.6352 - auc_2: 0.6904 - accuracy: 0.6386
Epoch 3/75
144/144 - 1s - loss: 0.6341 - auc_2: 0.6917 - accuracy: 0.6400
Epoch 4/75
144/144 - 1s - loss: 0.6335 - auc_2: 0.6926 - accuracy: 0.6397
Epoch 5/75
144/144 - 1s - loss: 0.6329 - auc_2: 0.6934 - accuracy: 0.6409
Epoch 6/75
144/144 - 1s - loss: 0.6324 - auc_2: 0.6941 - accuracy: 0.6413
Epoch 7/75
144/144 - 0s - loss: 0.6320 - auc_2: 0.6947 - accuracy: 0.6418
Epoch 8/75
144/144 - 0s - loss: 0.6316 - auc_2: 0.6951 - accuracy: 0.6422
Epoch 9/75
144/144 - 0s - loss: 0.6312 - auc_2: 0.6957 - accuracy: 0.6427
Epoch 10/75
144/144 - 0s - loss: 0.6307 - auc_2: 0.6963 - accuracy: 0.6426
Epoch 11/75
144/144 - 1s - loss: 0.6304 - auc_2: 0.6969 - accuracy: 0.6432
Epoch 12/75
144/144 - 1s - loss: 0.6300 - auc_2: 0.6974 - accuracy: 0.6436
Epoch 13/75
144/144 - 0s - loss: 0.6296 - auc_2: 0.6979 - accuracy: 0.6440
Epoch 14/75
144/144 - 0s - loss: 0

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 25.6min finished


Epoch 1/75
703/703 - 3s - loss: 0.4016 - auc_2: 0.7559 - accuracy: 0.8456
Epoch 2/75
703/703 - 3s - loss: 0.3968 - auc_2: 0.6941 - accuracy: 0.8468
Epoch 3/75
703/703 - 3s - loss: 0.3961 - auc_2: 0.6959 - accuracy: 0.8468
Epoch 4/75
703/703 - 3s - loss: 0.3956 - auc_2: 0.6972 - accuracy: 0.8468
Epoch 5/75
703/703 - 3s - loss: 0.3952 - auc_2: 0.6983 - accuracy: 0.8469
Epoch 6/75
703/703 - 3s - loss: 0.3949 - auc_2: 0.6991 - accuracy: 0.8469
Epoch 7/75
703/703 - 2s - loss: 0.3947 - auc_2: 0.6997 - accuracy: 0.8469
Epoch 8/75
703/703 - 3s - loss: 0.3945 - auc_2: 0.7003 - accuracy: 0.8469
Epoch 9/75
703/703 - 3s - loss: 0.3942 - auc_2: 0.7011 - accuracy: 0.8470
Epoch 10/75
703/703 - 3s - loss: 0.3941 - auc_2: 0.7012 - accuracy: 0.8470
Epoch 11/75
703/703 - 3s - loss: 0.3940 - auc_2: 0.7016 - accuracy: 0.8470
Epoch 12/75
703/703 - 2s - loss: 0.3938 - auc_2: 0.7021 - accuracy: 0.8470
Epoch 13/75
703/703 - 2s - loss: 0.3938 - auc_2: 0.7022 - accuracy: 0.8470
Epoch 14/75
703/703 - 3s - loss: 0

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=0, shuffle=True),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('under',
                                        RandomUnderSampler(random_state=None,
                                                           ratio=None,
                                                           replacement=False,
                                                           return_indices=False,
                                                           sampling_strategy='auto')),
                                       ('model',
                                        <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f1c8b2f6128>)],
                                verb...
                      'average_precision': make_scorer(average_precision_score, needs_proba=True),
                      'balanced_accuracy': make_scorer(balanced_accuracy_score),
     

In [95]:
#Assign results of cross validation.
results_Grid3_NN = gridNNFinal.cv_results_

In [96]:
#Best parameters based on loan loss:
ind = np.argmin(results_Grid3_NN['rank_test_loan_loss'])
print('Parameters:', results_Grid3_NN['params'][ind])
print('ROC-AUC:', results_Grid3_NN['mean_test_roc_auc'][ind])
print('F1:', results_Grid3_NN['mean_test_f1'][ind])
print('Accuracy:', results_Grid3_NN['mean_test_accuracy'][ind])
print('Balanced Accuracy:', results_Grid3_NN['mean_test_balanced_accuracy'][ind])
print('Loan Loss:', results_Grid3_NN['mean_test_loan_loss'][ind])

Parameters: {'model__batch_size': 1000, 'model__epochs': 75, 'under': RandomUnderSampler(random_state=0, ratio=None, replacement=False,
                   return_indices=False, sampling_strategy=0.6)}
ROC-AUC: 0.6953844693161911
F1: 0.33445862103763235
Accuracy: 0.765853066863268
Balanced Accuracy: 0.6095152393170457
Loan Loss: -114485.96666666667


In [ ]:
#Best parameters based on loan loss:
ind = np.argmin(results_Grid3_NN['rank_test_loan_loss'])
print('Parameters:', results_Grid3_NN['params'][ind])
print('ROC-AUC:', results_Grid3_NN['mean_test_roc_auc'][ind])
print('F1:', results_Grid3_NN['mean_test_f1'][ind])
print('Accuracy:', results_Grid3_NN['mean_test_accuracy'][ind])
print('Balanced Accuracy:', results_Grid3_NN['mean_test_balanced_accuracy'][ind])
print('Loan Loss:', results_Grid3_NN['mean_test_loan_loss'][ind])

Parameters: {'model__batch_size': 1000, 'model__epochs': 75, 'under': RandomUnderSampler(random_state=0, ratio=None, replacement=False,
                   return_indices=False, sampling_strategy=0.7)}
ROC-AUC: 0.6953253375609237
F1: 0.3475188306723995
Accuracy: 0.7353731171588599
Balanced Accuracy: 0.6227642051630206
Loan Loss: -114235.3


# Final NN Model

In [138]:
#Setup pipeline.
stepsNN = [('under', RandomUnderSampler(random_state=rs, sampling_strategy=0.7)), ('model', KerasClassifier(build_fn=create_model2, verbose=2))]
pipelineNN = Pipeline(steps=stepsNN)

In [ ]:
#Train pipeline. Orig
pipelineNN.fit(X=dfPostImpute[trainAll], y = trainLabels, model__batch_size = 1000, model__epochs = 75)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Epoch 1/75
262/262 - 1s - loss: 0.6295 - auc: 0.6769 - accuracy: 0.6405
Epoch 2/75
262/262 - 1s - loss: 0.6207 - auc: 0.6911 - accuracy: 0.6499
Epoch 3/75
262/262 - 1s - loss: 0.6195 - auc: 0.6929 - accuracy: 0.6505
Epoch 4/75
262/262 - 1s - loss: 0.6187 - auc: 0.6941 - accuracy: 0.6521
Epoch 5/75
262/262 - 1s - loss: 0.6180 - auc: 0.6952 - accuracy: 0.6519
Epoch 6/75
262/262 - 1s - loss: 0.6172 - auc: 0.6963 - accuracy: 0.6531
Epoch 7/75
262/262 - 1s - loss: 0.6167 - auc: 0.6972 - accuracy: 0.6530
Epoch 8/75
262/262 - 1s - loss: 0.6161 - auc: 0.6980 - accuracy: 0.6542
Epoch 9/75
262/262 - 1s - loss: 0.6156 - auc: 0.6987 - accuracy: 0.6541
Epoch 10/75
262/262 - 1s - loss: 0.6152 - auc: 0.6993 - accuracy: 0.6551
Epoch 11/75
262/262 - 1s - loss: 0.6150 - auc: 0.6997 - accuracy: 0.6556
Epoch 12/75
262/262 - 1s - loss: 0.6147 - auc: 0.7000 - accuracy: 0.6551
Epoch 13/75
262/262 - 1s - loss: 0.6143 - auc: 0.7006 - accuracy: 0.6560
Epoch 14/75
262/262 - 1s - loss: 0.6142 - auc: 0.7009 - accu

Pipeline(memory=None,
         steps=[('under',
                 RandomUnderSampler(random_state=0, ratio=None,
                                    replacement=False, return_indices=False,
                                    sampling_strategy=0.7)),
                ('model',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff5dbe4d5c0>)],
         verbose=False)

In [140]:
#Predict class labels
y_hat_NN = pipelineNN.predict(dfPostImpute[testAll])

11429/11429 - 10s


In [141]:
#Predict probabilities
y_prob_NN = pipelineNN.predict_proba(dfPostImpute[testAll])

11429/11429 - 10s


In [142]:
#Unpack class labels into an array
y_hat_NN = np.array([i[0] for i in y_hat_NN])

In [146]:
#Create a series with predictions.
resultsNN = {'id': dfFinal[testAll].id.values, 'true_class': testLabels, 'predict_class_NN': y_hat_NN, 
                 'prob_NN':y_prob_NN[:,1]}

In [147]:
#Convert series with predictions to dataframe.
df_Results_NN = pd.DataFrame(resultsNN)

In [148]:
#Save dataframe to csv file.
df_Results_NN.to_csv('results_NN_New.csv')